In [ ]:
import pandas as pd
import random

# Padrão de faixa etária do Brasil
faixas_etarias = [0, 10, 12, 14, 16, 18]
#sendo 0-LIVRE PARA TODOS OS PUBLICOS,
# 10- MAIORES DE 10 ANOS,
# 12- MAIORES DE 12 ANOS,
# 14- MAIORES DE 14 ANOS,
# 16- MAIORES DE 16 ANOS,
# 18- MAIORES DE 18 ANOS

canais_marketing = ['Instagram', 'Google Ads', 'Twitter', 'Facebook', 'LinkedIn', 'YouTube']

def gerar_dados_ficticios(n):
    dados = []
    for _ in range(n):
        nome = f"Campanha_{random.randint(1, 10000)}"
        titulo = f"Filme_{random.randint(1, 10000)}"
        faixa_etaria = random.choice(faixas_etarias)
        budget_total = round(random.uniform(1000, 50000), 2)
        expectativa_alcance = random.randint(10000, 1000000)
        canal_marketing = random.choice(canais_marketing)
        roi = round(random.uniform(0.5, 10) * budget_total, 2)
        cpc = round(random.uniform(0.1, 10), 2)
        numero_conversoes = int(expectativa_alcance * random.uniform(0.01, 0.2))

        dados.append([
            nome, titulo, faixa_etaria, budget_total,
            expectativa_alcance, canal_marketing, roi, cpc, numero_conversoes
        ])

    return pd.DataFrame(dados, columns=[
        'nome', 'titulo', 'faixa_etaria', 'budget_total',
        'expectativa_alcance', 'canal_marketing', 'roi', 'cpc', 'numero_conversoes'
    ])

# Gerar o dataset
dataset = gerar_dados_ficticios(10000)
dataset.to_csv('dataset_campanhas.csv', index=False)
display(dataset)

,nome,titulo,faixa_etaria,budget_total,expectativa_alcance,canal_marketing,roi,cpc,numero_conversoes
0,Campanha_5643,Filme_113,10,27602.90,16190,Google Ads,207383.76,3.79,3092
1,Campanha_9600,Filme_9748,14,11227.51,255117,Google Ads,94959.43,7.96,29569
2,Campanha_5288,Filme_5490,10,43232.08,787949,Instagram,257388.98,2.85,88127
3,Campanha_8723,Filme_3566,10,3898.10,812942,YouTube,38778.04,9.87,150058
4,Campanha_1528,Filme_1106,16,30083.97,557792,Facebook,78442.49,4.50,26757
...,...,...,...,...,...,...,...,...,...
9995,Campanha_8727,Filme_9104,10,31864.83,21679,YouTube,262648.20,8.92,1838
9996,Campanha_4346,Filme_6394,10,49574.36,847024,YouTube,315243.38,1.98,150845
9997,Campanha_381,Filme_6977,10,31365.44,657519,YouTube,112148.20,6.16,86227
9998,Campanha_2826,Filme_5417,12,3558.04,48008,LinkedIn,25274.77,1.69,4040


# Teste 1

In [ ]:
# -----------------------------------------------
# -------- MODELO PREDITIVO CANAL MARKETING -----
# -----------------------------------------------

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Carregar o dataset
dataset = pd.read_csv('dataset_campanhas.csv')

# Remover colunas irrelevantes
dataset = dataset.drop(['nome', 'titulo'], axis=1)

# Mapeamento dos canais de marketing para números
canal_marketing_mapping = {
    'Instagram': 0,
    'Google Ads': 1,
    'Twitter': 2,
    'Facebook': 3,
    'LinkedIn': 4,
    'YouTube': 5
}

dataset['canal_marketing'] = dataset['canal_marketing'].map(canal_marketing_mapping)

# Separar as variáveis independentes e dependentes
X = dataset.drop('canal_marketing', axis=1)
y = dataset['canal_marketing']

# Dividir o dataset em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinar o modelo
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Fazer previsões
y_pred = model.predict(X_test)

# Avaliar o modelo
accuracy = accuracy_score(y_test, y_pred)
accuracy_percentage = accuracy * 100

# Reverter a codificação numérica para os nomes dos canais para o relatório
inv_canal_marketing_mapping = {v: k for k, v in canal_marketing_mapping.items()}
target_names = [inv_canal_marketing_mapping[i] for i in sorted(canal_marketing_mapping.values())]

report = classification_report(y_test, y_pred, target_names=target_names)

print(f"Acurácia do modelo: {accuracy_percentage:.2f}%")
print("Relatório de Classificação:")
print(report)


Acurácia do modelo: 16.95%
Relatório de Classificação:
              precision    recall  f1-score   support

   Instagram       0.17      0.17      0.17       328
  Google Ads       0.14      0.16      0.15       316
     Twitter       0.18      0.21      0.19       335
    Facebook       0.21      0.21      0.21       357
    LinkedIn       0.18      0.15      0.16       334
     YouTube       0.14      0.12      0.13       330

    accuracy                           0.17      2000
   macro avg       0.17      0.17      0.17      2000
weighted avg       0.17      0.17      0.17      2000



In [ ]:
# -----------------------------------------------
# -------- MODELO PREDITIVO CANAL MARKETING -----
# -----------------------------------------------

import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

# Carregar o dataset
dataset = pd.read_csv('dataset_campanhas.csv')

# Pré-processamento
# Codificar variáveis categóricas (canal_marketing)
le = LabelEncoder()
dataset['canal_marketing'] = le.fit_transform(dataset['canal_marketing'])

# Remover colunas não numéricas que não são úteis para a predição
dataset = dataset.drop(columns=['nome', 'titulo'])

# Criar novas features (opcional)
# Exemplo: relação entre budget e número de conversões
dataset['budget_per_conversion'] = dataset['budget_total'] / (dataset['numero_conversoes'] + 1)  # +1 para evitar divisão por zero

# Dividir em features (X) e target (y)
X = dataset.drop(columns=['canal_marketing'])
y = dataset['canal_marketing']

# Normalizar as features numéricas
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Dividir em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Testar diferentes valores de k para KNN
best_k = 0
best_score = 0
for k in range(3, 21, 2):
    model = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(model, X_train, y_train, cv=5)
    mean_score = np.mean(scores)
    print(f"Acurácia média com {k} vizinhos: {mean_score:.4f}")
    if mean_score > best_score:
        best_score = mean_score
        best_k = k

print(f"\nMelhor valor de k: {best_k} com acurácia média de {best_score:.4f}")

# Treinar o modelo final com o melhor k
model = KNeighborsClassifier(n_neighbors=best_k)
model.fit(X_train, y_train)

# Fazer previsões
y_pred = model.predict(X_test)

# Avaliar o modelo
accuracy = accuracy_score(y_test, y_pred)
accuracy_percentage = accuracy * 100
report = classification_report(y_test, y_pred, target_names=label_encoders['canal_marketing'].classes_)

print(f"Acurácia do modelo: {accuracy_percentage:.2f}%")
print("Relatório de Classificação:")
print(report)

Acurácia média com 3 vizinhos: 0.1659
Acurácia média com 5 vizinhos: 0.1633
Acurácia média com 7 vizinhos: 0.1596
Acurácia média com 9 vizinhos: 0.1654
Acurácia média com 11 vizinhos: 0.1671
Acurácia média com 13 vizinhos: 0.1660
Acurácia média com 15 vizinhos: 0.1653
Acurácia média com 17 vizinhos: 0.1624
Acurácia média com 19 vizinhos: 0.1614

Melhor valor de k: 11 com acurácia média de 0.1671
Acurácia do modelo: 16.00%
Relatório de Classificação:
              precision    recall  f1-score   support

    Facebook       0.17      0.26      0.21       515
  Google Ads       0.14      0.16      0.15       511
   Instagram       0.17      0.19      0.18       483
    LinkedIn       0.16      0.13      0.14       495
     Twitter       0.14      0.11      0.13       487
     YouTube       0.17      0.11      0.13       509

    accuracy                           0.16      3000
   macro avg       0.16      0.16      0.16      3000
weighted avg       0.16      0.16      0.16      3000



In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Carregar o dataset
dataset = pd.read_csv('dataset_campanhas.csv')

# Remover colunas irrelevantes
dataset = dataset.drop(['nome', 'titulo'], axis=1)

# Mapeamento dos canais de marketing para números (se ainda não foi feito)
canal_marketing_mapping = {
    'Instagram': 0,
    'Google Ads': 1,
    'Twitter': 2,
    'Facebook': 3,
    'LinkedIn': 4,
    'YouTube': 5
}
dataset['canal_marketing'] = dataset['canal_marketing'].map(canal_marketing_mapping)

# Separar as variáveis independentes e a variável dependente (ROI)
X = dataset.drop('roi', axis=1)
y = dataset['roi']

# Dividir o dataset em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinar o modelo
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Fazer previsões
y_pred = model.predict(X_test)

# Avaliar o modelo
mse = mean_squared_error(y_test, y_pred)
rmse = mse**0.5
r2 = r2_score(y_test, y_pred)

print(f"Erro Quadrático Médio (MSE): {mse:.2f}")
print(f"Raiz do Erro Quadrático Médio (RMSE): {rmse:.2f}")
print(f"Coeficiente de Determinação (R²): {r2:.2f}")


Erro Quadrático Médio (MSE): 7608012443.59
Raiz do Erro Quadrático Médio (RMSE): 87223.92
Coeficiente de Determinação (R²): 0.35


In [ ]:
# ----------------------------------------------------
# -------- MODELO PREDITIVO ROI, CPC e Conversão -----
# ----------------------------------------------------

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Carregar o dataset
dataset = pd.read_csv('dataset_campanhas.csv')

# Pré-processamento
# Codificar variáveis categóricas (canal_marketing)
le = LabelEncoder()
dataset['canal_marketing'] = le.fit_transform(dataset['canal_marketing'])

# Remover colunas não numéricas que não são úteis para a predição
dataset = dataset.drop(columns=['nome', 'titulo'])

# Criar novas features (opcional)
dataset['budget_per_conversion'] = dataset['budget_total'] / (dataset['numero_conversoes'] + 1)  # +1 para evitar divisão por zero

# Normalizar as features numéricas
scaler = StandardScaler()
X_scaled = scaler.fit_transform(dataset.drop(columns=['roi', 'cpc', 'numero_conversoes']))

# Dividir em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_scaled, dataset[['roi', 'cpc', 'numero_conversoes']], test_size=0.3, random_state=42)

# Função para treinar e avaliar um modelo
def train_and_evaluate(model, X_train, y_train, X_test, y_test, target_name):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"\n{target_name} - {model.__class__.__name__}")
    print(f"Mean Squared Error: {mse:.4f}")
    print(f"R^2 Score: {r2:.4f}")
    return y_pred

# Modelos para ROI, CPC e número de conversões
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(alpha=1.0),
    'Random Forest Regressor': RandomForestRegressor(n_estimators=100, random_state=42)
}

# Treinar e avaliar modelos para cada alvo
for target in ['roi', 'cpc', 'numero_conversoes']:
    print(f"\n### Prevendo {target.upper()} ###")
    for model_name, model in models.items():
        train_and_evaluate(model, X_train, y_train[target], X_test, y_test[target], target)



### Prevendo ROI ###

roi - LinearRegression
Mean Squared Error: 6259108407.3160
R^2 Score: 0.4760

roi - Ridge
Mean Squared Error: 6259162954.8494
R^2 Score: 0.4760

roi - RandomForestRegressor
Mean Squared Error: 6707049516.9081
R^2 Score: 0.4385

### Prevendo CPC ###

cpc - LinearRegression
Mean Squared Error: 8.0845
R^2 Score: 0.0003

cpc - Ridge
Mean Squared Error: 8.0845
R^2 Score: 0.0003

cpc - RandomForestRegressor
Mean Squared Error: 8.6842
R^2 Score: -0.0739

### Prevendo NUMERO_CONVERSOES ###

numero_conversoes - LinearRegression
Mean Squared Error: 996813507.2676
R^2 Score: 0.4769

numero_conversoes - Ridge
Mean Squared Error: 996818202.5487
R^2 Score: 0.4769

numero_conversoes - RandomForestRegressor
Mean Squared Error: 8557923.7478
R^2 Score: 0.9955


In [ ]:
# -------------------------------------------------------
# -------- MODELO PREDITIVO ROI - Linear Regression -----
# -------------------------------------------------------

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Carregar o dataset
dataset = pd.read_csv('dataset_campanhas.csv')

# Pré-processamento
le = LabelEncoder()
dataset['canal_marketing'] = le.fit_transform(dataset['canal_marketing'])
dataset = dataset.drop(columns=['nome', 'titulo'])
dataset['budget_per_conversion'] = dataset['budget_total'] / (dataset['numero_conversoes'] + 1)

# Normalizar as features numéricas
scaler = StandardScaler()
X_scaled = scaler.fit_transform(dataset.drop(columns=['roi']))

# Dividir em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_scaled, dataset['roi'], test_size=0.3, random_state=42)

# Treinar e avaliar o modelo Linear Regression
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Avaliação do modelo
print("### Prevendo ROI ###")
print(f"Mean Squared Error: {mean_squared_error(y_test, y_pred):.4f}")
print(f"R^2 Score: {r2_score(y_test, y_pred):.4f}")


### Prevendo ROI ###
Mean Squared Error: 6260260836.3571
R^2 Score: 0.4759


In [ ]:
# -------------------------------------------------------
# -------- MODELO PREDITIVO CPC - Linear Regression -----
# -------------------------------------------------------

In [ ]:
# ---------------------------------------------------------
# -------- MODELO PREDITIVO Conversão - Random Forest -----
# ---------------------------------------------------------

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Carregar o dataset
dataset = pd.read_csv('dataset_campanhas.csv')

# Pré-processamento
le = LabelEncoder()
dataset['canal_marketing'] = le.fit_transform(dataset['canal_marketing'])
dataset = dataset.drop(columns=['nome', 'titulo'])
dataset['budget_per_conversion'] = dataset['budget_total'] / (dataset['numero_conversoes'] + 1)

# Normalizar as features numéricas
scaler = StandardScaler()
X_scaled = scaler.fit_transform(dataset.drop(columns=['numero_conversoes']))

# Dividir em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_scaled, dataset['numero_conversoes'], test_size=0.3, random_state=42)

# Treinar e avaliar o modelo Random Forest Regressor
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Avaliação do modelo
print("### Prevendo Número de Conversões ###")
print(f"Mean Squared Error: {mean_squared_error(y_test, y_pred):.4f}")
print(f"R^2 Score: {r2_score(y_test, y_pred):.4f}")



### Prevendo Número de Conversões ###
Mean Squared Error: 10691216.7396
R^2 Score: 0.9944


# Teste 2